In [27]:
import numpy as np

X = np.array([
  [1,1,0,1],
  [1,1,1,1],
  [1,0,1,1],
  [0,0,0,1],
])
                
y = np.array([[1],[0],[1],[0],])

In [36]:
np.random.seed(1)
class NN:
  def __init__(self, X, y, hidden_layers, loss_function='mae'):
    self.X = np.array(X)
    self.y = np.array(y)
    self.y = self.y.reshape(self.y.shape[0], 1)
    self.schema = [X.shape[1]] + hidden_layers + [self.y.shape[1]]
    self.schema_len = range(len(self.schema[:-1]))
    self.loss_function = loss_function
    for i, layer in enumerate(self.schema[:-1]):
      setattr(self, f'W{i}', np.random.uniform(-1, 1, (layer, self.schema[i+1])))

      
  def nonlin(self, x, deriv=False):
    if(deriv==True):
      return x * (1 - x)
    return 1/ (1 + np.exp(-x))
  
  
  def forward(self, X=None):
    if X:
      self.l0 = np.array(X)
    else:
      self.l0 = self.X
    for i in self.schema_len:
      l, W = (getattr(self, j) for j in f'l{i} W{i}'.split())
      setattr(self, f'l{i+1}', self.nonlin(l.dot(W)))
    return getattr(self, f'l{len(self.schema_len)}')
    
  
  def loss(self, error):
    loss_dict = {
      'mse': lambda x: np.mean(x**2),
      'mae': lambda x: np.mean(abs(x)),
    }
    loss_function = loss_dict[self.loss_function]
    return loss_function(error)
    
  def backward(self):

    # calculation loss
    for i in reversed(self.schema_len):
      i += 1
      l = getattr(self, f'l{i}')
      if i == len(self.schema_len):
        error = self.y - l
        self.error = self.loss(error)
      else:
        delta, W = (getattr(self, j) for j in f'l{i}delta W{i}'.split())
        error = delta.dot(W.T)
      setattr(self, f'l{i-1}delta', error * self.nonlin(l, deriv=True))
      
    # adjusting weights
    for i in reversed(self.schema_len):
      l, W, delta = (getattr(self, j) for j in f'l{i} W{i} l{i}delta'.split())
      setattr(self, f'W{i}', W + l.T.dot(delta))
    
    
  def train(self, epochs=1000, print_nth_epoch=100):
    for j in range(epochs):
      self.forward()
      self.backward()
      if print_nth_epoch and not j % print_nth_epoch:
        print(f'Error: {self.error}')
       
      
  def predict(self, x):
    return self.forward(x)

In [37]:
nn = NN(X, y, [100,50,25,10])

In [38]:
nn.train()

Error: 0.4978693614194011
Error: 0.23589051861677926
Error: 0.03786286917617938
Error: 0.024881955673188207
Error: 0.019620739046420657
Error: 0.01662329092034298
Error: 0.014637288732046538
Error: 0.013202483571224569
Error: 0.012105816156958467
Error: 0.01123368439242848


In [39]:
nn.predict([[1,0,0,1], [1,1,1,0], [0,1,1,0]])

array([[0.9945949 ],
       [0.02733906],
       [0.00453795]])

In [40]:
nn.loss_function

'mae'